# Part 1: Gate Fidelity Computation

In [ ]:
# Make sure that the installations in env.yaml are correct

import qutip as qt
import scqubits as scq
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import itertools
import os
from pathos.multiprocessing import ProcessingPool as Pool
import warnings

Define the Fluxonium Qubit

In [ ]:
fluxonium = scq.Fluxonium(EJ = 8.9,
                               EC = 2.5,
                               EL = 0.5,
                               flux = 0.48,
                               cutoff = 110)

levels = 6 # can be changed to include more or less energy levels 

evals, evecs = fluxonium.eigensys(evals_count=levels)

n_op_energy_basis = qt.Qobj(fluxonium.process_op(fluxonium.n_operator(), energy_esys=(evals, evecs)))

H0 = qt.Qobj(np.diag(evals)) * 2 * np.pi  # in GHz

A = 0.4 * 2 * np.pi  # drive amplitude in GHz
drive_op = n_op_energy_basis

omega_d = (evals[1] - evals[0]) * 2 * np.pi  # resonant drive frequency in GHz

H = [H0, [A * drive_op, 'cos(wd * t)']]
args = {'wd': omega_d}

#drive_op

In [ ]:
print("drive_op is Hermitian:", drive_op.isherm)

In [ ]:
psi_initial = qt.basis(levels, 0)

# psi_initial

In [ ]:
projectors = []

for i in range(levels):
    proj = qt.basis(levels, i) * qt.basis(levels, i).dag()
    projectors.append(proj)

# projectors

Time range defined

In [ ]:
iterations = 10000

times = np.linspace(0, 1000, iterations)

options = qt.Options(nsteps=1000000, store_states=True)

result_levels = qt.mesolve(
    H, psi_initial, times, [],
    projectors,
    args=args, options=options
)

# result_levels

In [ ]:
for i in range(levels):
    plt.plot(times, result_levels.expect[i], label=f'Population |{i}>')

plt.xlabel('Time')
plt.ylabel('Population')
plt.title('Time Evolution of Level Populations for X Gate')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()  
plt.show()

In [ ]:
rabi_freq = A * abs(n_op_energy_basis[0, 1])

tx = np.pi / rabi_freq

# tx

In [ ]:
x_test_times = np.linspace(0, tx, 1000) 

result_levels_x_time = qt.mesolve(
    H, psi_initial, x_test_times, [],
    projectors,
    args=args, options=options
)

for i in range(levels):
    plt.plot(x_test_times, result_levels_x_time.expect[i], label=f'Population |{i}>')

plt.xlabel('Time')
plt.ylabel('Population')

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()  
plt.show()

# Part 2: Fidelity

In [ ]:
X_ideal = qt.sigmax()

X_ideal

In [ ]:
# example propagator for time tx

U = qt.propagator(H, tx, args=args, options=options)

U

In [ ]:
options = qt.Options(nsteps=1000000, store_states=True, atol=1e-12, rtol=1e-11)

print("Calculating propagators...")
P_list = qt.propagator(H, times, args=args, options=options)

fid_times = []
fid_values = []
for i, t in enumerate(tqdm(times, desc='Calculating fidelity')):
    U_t = P_list[i]
    U_2lvl = qt.Qobj(U_t.full()[:2, :2])
    fid = qt.average_gate_fidelity(U_2lvl, X_ideal)
    
    fid_times.append(t)
    fid_values.append(fid)

plt.plot(np.array(fid_times), np.array(fid_values))
plt.xlabel('Time')
plt.ylabel('Average Gate Fidelity')
plt.title('Average Gate Fidelity vs Time')
plt.show()

In [ ]:
peak_fidelity = np.max(fid_values)
print("Peak average gate fidelity:", peak_fidelity)

peak_index = np.argmax(fid_values)
peak_time = times[peak_index]
print("Time at peak fidelity:", peak_time)

tx = peak_time

# Part 3: Factoring in Noise

In [ ]:
gamma_ij = {}
for j in range(1, levels):
    for i in range(j):
        t1 = fluxonium.t1_capacitive(j, i, Q_cap=1e5)
        if t1 is not None and t1 > 0:
            rate = 1.0 / t1
            gamma_ij[(i, j)] = rate
            gamma_ij[(j, i)] = rate  
gamma_ij

In [ ]:
c_ops = []
for (i, j), gamma in gamma_ij.items():
    # |i><j| operator
    cop = (np.sqrt(gamma)) * qt.basis(levels, i) * qt.basis(levels, j).dag()
    c_ops.append(cop)

# c_ops

## Propagators

In [ ]:
# times = np.linspace(0, 1000, iterations)

print("Calculating propagators...")
P_list = qt.propagator(H, times, c_ops=c_ops, args=args, options=options)

In [ ]:
p_unitary = []

for i in range (len(P_list)):

    p_special = P_list[i]

    P_kraus_special = qt.to_kraus(p_special)

    p_special_2x2 = [qt.Qobj(k.full()[:2, :2]) for k in P_kraus_special]

    p_special_2x2_super = qt.kraus_to_super(p_special_2x2)

    p_unitary.append(p_special_2x2_super)

In [ ]:
fid_times = []
fid_values = []

for i, t in enumerate(tqdm(times, desc='Calculating fidelity')):
    # U_t = P_list[i]

    #testing
    fid = qt.average_gate_fidelity(p_unitary[i], X_ideal)
    
    fid_times.append(t)
    fid_values.append(fid)

plt.plot(np.array(fid_times), np.array(fid_values))
plt.xlabel('Time')
plt.ylabel('Average Gate Fidelity')
plt.title('Average Gate Fidelity vs Time')
plt.show()

In [ ]:
peak_fidelity_noise = np.max(fid_values)
print("Peak average gate fidelity:", peak_fidelity_noise)

peak_index_noise = np.argmax(fid_values)
peak_time_noise = times[peak_index_noise]
print("Time at peak fidelity:", peak_time_noise) #should be a bit different than the peak time without noise

# Part 4: Optimization (Serial)

In [ ]:
print("Drive frequency (GHz):", omega_d)

In [ ]:
omega_d_array = np.linspace(0.8 * omega_d, 1.2 * omega_d, 4) #note that the 4 can be changed to increase the number of points

omega_d_array

In [ ]:
print("Peak time", peak_time)

print("Peak time with noise", peak_time_noise)

t_g_array = np.linspace(0.8 * peak_time_noise, 1.2 * peak_time_noise, 4)

t_g_array

In [ ]:
param_pairs = list(itertools.product(omega_d_array, t_g_array))

# param_pairs

In [ ]:
# special function definitions

from import_functions import param_map, parallel_map_qutip, evolve, evolve_wrapped

In [ ]:
try:
    # pathos implementation is much more robust - should install if not present
    import pathos.multiprocessing as mp
except ImportError:
    # but default to std library version
    print(
        "using std lib version of multiprocessing; consider installing pathos; it's much more robust"
    )
    import multiprocessing as mp

In [ ]:
param_pairs = list(itertools.product(omega_d_array, t_g_array))

results_flat = parallel_map_qutip_cleaned(evolve_wrapped, param_pairs, num_cpus=4)
results = np.reshape(results_flat, (len(omega_d_array), len(t_g_array)))


In [ ]:
# print the results with the command below if desired

# results

In [ ]:
# Find the indices of the maximum fidelity in the results array
max_idx = np.unravel_index(np.argmax(results), results.shape)
max_value = results[max_idx]
omega_d_best = omega_d_array[max_idx[0]]
t_g_best = t_g_array[max_idx[1]]

print(f"Best fidelity: {max_value}")
print(f"Found at omega_d = {omega_d_best}, t_g = {t_g_best}")
print(f"Indices in results array: {max_idx}")

# Part 5: Optimization (Parallel)

In [ ]:
# Job initialization

evals, _ = fluxonium.eigensys(evals_count=levels)
omega_d_base = (evals[1] - evals[0]) * 2 * np.pi #in case not run in cell above

print("omega d is", omega_d_base)

# can modify the dimensions to increase/decrease the number of points in the sweep
dimensions_omega = 8
dimensions_time = 7


# note that the ranges can be modified to sweep a larger or smaller range
# the only reason these numbers are displayed is to exhibit the exact ranges
# used in the poster
omega_d_array = np.linspace(omega_d_base - 0.01, omega_d_base + 0.01, dimensions_omega)

t_g_array = np.linspace(24.2, 24.6, dimensions_time)
# param_pairs = list(itertools.product(omega_d_array, t_g_array))

# print(f"Total simulations to run: {len(param_pairs)}")

warnings.filterwarnings(
    "ignore",
    module="qutip.*"  # Regex pattern to match all warnings from qutip
)
scq.settings.T1_DEFAULT_WARNING=False

In [ ]:
fidelity_results = param_map(evolve, [omega_d_array, t_g_array], map_fun=parallel_map_qutip)

In [ ]:
# printing convenience

print("t g array is ", t_g_array)
print("omega d array is ", omega_d_array)
print("Omega by time dimensions are ", dimensions_omega, " by ", dimensions_time)
print("fidelity results are ", fidelity_results)

## Heat Map plotting

In [ ]:
from import_functions import density

fidelity_results = np.asarray(fidelity_results,dtype = float)

density(fidelity_results, omega_d_array, t_g_array)